## Python for REU 2019

_Burt Rosenberg, 12 May 2019_


### Various thing about lists



The list is one of the most versatile datastructures. It models a sequence of items, which can be a mix of types.

Items can be accessed through the index of the item on the list. The first item is at index 0. For a list L access the i-th item with square brackets: L[i]. The last item on the list is at index -1, and the previous to last, at -2, and so on.

**Slices**

Python supports the selection of multiple elements by index using *slice* notation. The full notation is [a:b:c] fo for a slace beginning at a, ending before b, advancing to include only every c-th element.

Slice notation can appear on the right hand side of an assignment, to retrieve a slice of elements, or on the left hand side of an assignment, to receive a slice of elements.

Leaving a location blank refers to the default (but see the discussion below on negative skips):

* the default for the start is the first item in the list
* the default for the end is the list item in the list
* the default for the skip is to take all items in the start-end range


Here are some fun tricks with slices:

In [37]:
def fun_slice():
    """
    here are some fun slice tricks.
    """
    a = [i for i in range(10)]
    print(a)
    print(a[0],a[-1])
    print(a[::2])
    print(a[::-2])

    b = [-i for i in range(10)]
    print(b)
    b[::2] = a[::2]
    print(b)
    b[::2] = a[::-2]
    print(b)
    
    a[len(a)//2:] = a[len(a)//2::-1]
    print(a)
    
fun_slice()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
0 9
[0, 2, 4, 6, 8]
[9, 7, 5, 3, 1]
[0, -1, -2, -3, -4, -5, -6, -7, -8, -9]
[0, -1, 2, -3, 4, -5, 6, -7, 8, -9]
[9, -1, 7, -3, 5, -5, 3, -7, 1, -9]
[0, 1, 2, 3, 4, 5, 4, 3, 2, 1, 0]


__Shallow Copies__

There is an important difference between a slice of a list, and the list. A slice extracts values from the list and copies them over. 

We start with a simple example:

In [3]:
a = [i for i in range(10)]
b1 = a
b2 = a[:]
a[::2] = a[::-2]
print(a)
print(b1)
print(b2)

[9, 1, 7, 3, 5, 5, 3, 7, 1, 9]
[9, 1, 7, 3, 5, 5, 3, 7, 1, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


By using a slice to transfer values, the b2 array is not modified by changes in a, although b1 is. This is called a shallow copy, however, because changes in a can still change b2. If what is copied is a referene, rather than a value, changes in what the reference refers to is visible in the copy. For instance:

In [39]:
a = [[i] for i in range(10)]
b2 = a[:]
for i in range(len(a)): a[i][0] = -a[i][0]
print(a)
print(b2)

[[0], [-1], [-2], [-3], [-4], [-5], [-6], [-7], [-8], [-9]]
[[0], [-1], [-2], [-3], [-4], [-5], [-6], [-7], [-8], [-9]]


__Deep Copies__

A *deep copy* would descend recursively into each list and use the slice notation (among other methods) to copy values rather than references. The more complicated RememberMe class summarizes this.

In [40]:
class RememberMe:
    
    def __init__(self):
        pass
           
    def just_remember(self,o):
        self.that_object = o

    def copy_shallow(self,o):
        self.list_shallow = o[:]
 
    def copy_deep_aux(self,o):
        # if o is a list
        if isinstance(o,[].__class__):
            # shallow copy the backbone
            a = o[:]
            for i, b in enumerate(a):
                # and recurse on each element
                a[i] = self.copy_deep_aux(b)
            return a
        else:
            return o
            
    def copy_deep(self,o):
        self.list_deep = self.copy_deep_aux(o)      
        
    def tell_all(self):
        return (self.that_object,self.list_shallow,self.list_deep)

test_list = [1,2,3,[1,2,3],[[1,2,3],[1,2,3],[1,2,3]]]

r_m = RememberMe()
r_m.just_remember(test_list)
r_m.copy_shallow(test_list)
r_m.copy_deep(test_list)

test_list[0] = 'a'
test_list[3][0] = 'b'
test_list[4][0][0] = 'c'
(a,b,c) = r_m.tell_all()
print("saved the reference", a)
print("made a shallow copy", b)
print("made a deep copy", c)


saved the reference ['a', 2, 3, ['b', 2, 3], [['c', 2, 3], [1, 2, 3], [1, 2, 3]]]
made a shallow copy [1, 2, 3, ['b', 2, 3], [['c', 2, 3], [1, 2, 3], [1, 2, 3]]]
made a deep copy [1, 2, 3, [1, 2, 3], [[1, 2, 3], [1, 2, 3], [1, 2, 3]]]


__Slices with negative skip__

When slicing with a negative skip, the inclusion intervale changes polarity as well. The first in the colon separated triple is still the start index, and the second still indicates where the stop, but the slice will stop at one more than that value. Again, the interval goes until one before the ending index, but when descending "one before" is interpreted as ending index plus one, rather than minus one.

This is a good convention as it allows slices to concatenate without much math. For instance:

> a[:] = a[:i] + a[i:]


There are a couple subtilties in leaving one of the slice components empty. In fact, leaving it empty places in the component the value None.

>_None_ is a Python keyword that is the value, and only value, of the NoneType.

When None is in the start position, or the position is left empty, it means 0 for positive skip and -1 for negative skip. For the end position it means -1 for positive skip, but can not possibly mean any integer for a negative skip, and is the unique way to include the start of the array in this situation. Hence, the slice convention as two subtleties,

* None means different things according to skip direction
* None can address the beginning of the array in case of negative skip


In [19]:
test = [i for i in range(10)]

print(test[5:3:-1])
print(test[len(test):3:-1])
print(test[:3:-1])

# the reason for the "until before final" convention.
# it gives this simple equation:

test_glue = test[:len(test)//2] + test[len(test)//2:]
assert test == test_glue
print("assertion passed!")

def swap_at(a,i):
    """
    swaps a[i] and a[i+1]
    """
    j = i-1 if i>0 else None
    a[i:i+2] = a[i+1:j:-1]


def test_swap_at(n):
    test = [i for i in range(n)]
    for i in range(n):
        swap_at(test,i)
        print(test)
test_swap_at(5)


print(None.__class__.__name__)


[5, 4]
[9, 8, 7, 6, 5, 4]
[9, 8, 7, 6, 5, 4]
assertion passed!
[1, 0, 2, 3, 4]
[1, 2, 0, 3, 4]
[1, 2, 3, 0, 4]
[1, 2, 3, 4, 0]
[1, 2, 3, 4, 0]
NoneType


**Using a list as an iterator**

A list is iterable, so it can be used in a for statment exactly in place of the range() function.

It isn't exactly as if the list iterates, but that it wrap itself inside an iterator. If need be, you can explicitly implement to for loop by going from a list to an iterator with the iter function, then apply the next function to the iterator, and make sure that you catch the StopIteration exception.

To go from an iterator to a list, use the list function.

In [41]:
a_l = [i for i in range(10)]
for e in a_l:
    print(e)

a_l_itr = iter(a_l)
try:
    while True:
        print(next(a_l_itr))
except StopIteration:
    pass

a_l_itr = iter(a_l)
print(list(a_l_itr))

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


__some other list functions__

To find out more about list functions, look for the Python builtins 

* https://docs.python.org/3/library/functions.html 

and the list methods 

* https://docs.python.org/3/tutorial/datastructures.html

in the documentation. Some functions are both: sorted() is a builtin function and list.sorted() is a list method.


In [42]:
l = [3,6,9,1,2,7]

# min
print("min",min(l),"max",max(l))

#append
l.append(13)
print(l)

try:
    print([1,2,3].index(2))
    [1,2,3].index(4)
except ValueError:
    print("wasn't there")
    
try:
    l = [1,2,2,3]
    l.remove(2)
    print(l)
    [1,2,3].remove(4)
except ValueError:
    print("wasn't there")


min 1 max 9
[3, 6, 9, 1, 2, 7, 13]
1
wasn't there
[1, 2, 3]
wasn't there


### Exercise

A perfect shuffle is when the cards during a shuffle alternate perfectly: one from the left hand, one from the right hand, one from the left hand, and so on. Only a magician can perform a perfect shuffle, it requires such slight of hand. And only a mathematician can wonder what are the properties of a perfect shuffle? Does it, for instance, return the deck back to the original order after a certain, perhaps small, number of shuffles

The question was considered by someone who is both a mathematician and a magicina, Persi Diaconns.

In this exercise in slices, we will simulate a perfect shuffle and answer the question of how many perfect shuffles return the deck to the original order. The trick to the problem is to work backwards. Rather than considering a shuffle, simulate the un-shuffle, which would alternately take cards backwards from the finished shuffle, and stack the two halves and a backward cut.

In [43]:
# fix my broken code

def perfect_shuffle(deck):
    return deck

def n_perfect_shuffle(m):
    """
    answers the question: how many perfect shuffles on
    a deck of m cards returns the deck to the original order
    """
    deck = [i for i in range(m)]
    deck_org = deck[:]
    count = 0
    # a while loop until deck comes back to deck_org, perhaps?
    return count
    
def test_perfect_shuffle():
    ans = [0, 2, 4, 6, 1, 3, 5, 7]
    if perfect_shuffle([i for i in range(8)]) != ans:
        print("broken!")
        return
    
    # for a deck of 2^i cards, i perfect shuffles return the deck
    j = 8
    for i in range(3,6):
        if n_perfect_shuffle(j)!=i:
            print("broken!")
            return
        j *= 2
        
    # how many perfects shuffles for an actual deck
    if n_perfect_shuffle(52)!=8:
        print("broken!")
    else:
        print("correct!")

test_perfect_shuffle()

broken!


__Challenge problem__ 

Smug the dragon is clever enough to use his tail for a 3 hand perfect shuffle. What are the results for this shuffle?
